In [12]:
# read configs
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))
from steps import conf

data_url = "https://liftingcast.com/meets/mvcjgjxtrlxi/results"

In [3]:
import polars as pl

In [13]:
reference_tables = "../reference-tables/usa-power-lifting-australia-tsf-team-champs-at-the-fort-2024.opl.csv"

df = pl.read_csv(reference_tables, skip_rows=5)

In [15]:
renamed_columns = {column: conf.camel_to_snake(column) for column in df.columns}

In [16]:
team_champ_df = df.rename(mapping=renamed_columns)

In [24]:
team_champ_df.select(["name", "birth_year", "weight_class_kg"]).glimpse()

Rows: 86
Columns: 3
$ name            <str> 'Natasha Viselka', 'Vinning Mak', 'Aikaterini Chorianopoulou', 'Carol Chew', 'Thuy Vy Pham', 'Claudia Coppola', 'Lina Tran', 'Kiara Sala', 'Bernadette Mammone', 'Megan Tsai'
$ birth_year      <str> '1995', '1995', '', '1978', '2000', '1995', '1999', '', '1985', '2000'
$ weight_class_kg <str> '48', '52', '52', '56', '56', '56', '56', '60', '60', '60'



In [19]:
raw_file_path = "../data/raw/openpowerlifting-latest.parquet"
raw_df = pl.read_parquet(raw_file_path)

In [31]:
raw_df.filter(pl.col("name") == "Natasha Viselka").groupby("weight_class_kg").agg(pl.max("total_kg"))

/var/folders/k8/wkzv3jzs2_1flxr71zn08mmh0000gq/T/ipykernel_97380/3516695818.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  raw_df.filter(pl.col("name") == "Natasha Viselka").groupby("weight_class_kg").agg(pl.max("total_kg"))


weight_class_kg,total_kg
str,f64
"""47""",272.5
"""48""",317.5
